<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/notebooks/2020_0525detecron2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- source https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=9_FzH13EjseR


In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
     |████████████████████████████████| 276kB 4.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=cd144f79e38cd8ef8b21c537e9647f603bd28f3c726acd9270d5964a6467dd65
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-4n2e0gus
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-4n2e0gus
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-lin

In [4]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
     |████████████████████████████████| 6.2MB 1.9MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post200513-cp36-none-any.whl size=40893 sha256=eb33698f29acc1819d84ce6f883fd5947bc9448c05cb590068342f98925331e6
  Stored in directory: /root/.cache/pip/wheels/a2/f6/77/551770c4b8cd75e9335cd0acf59c08d60a8684048b19da6702
Successfully built fvcore


In [0]:
import glob
import multiprocessing as mp
import os
import time
import cv2
import tqdm

import PIL.Image
from IPython.display import clear_output, Image, display

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [8]:
from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

from detectron2.demo.predictor import VisualizationDemo

ModuleNotFoundError: ignored

In [20]:
#!find /usr/local/lib/python3.6/dist-packages/detectron2/ -name 'predictor*'
!ls /usr/local/lib/python3.6/dist-packages/detectron2/config/


compat.py  config.py  defaults.py  __init__.py	__pycache__


In [0]:
def setup_cfg(config_file, opts, confidence_threshold=0.5):
    # load config from file and command-line arguments
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.merge_from_list(opts)
    # Set score_threshold for builtin models
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
    cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = confidence_threshold
    cfg.freeze()
    return cfg


In [24]:
base = '/usr/local/lib/python3.6/dist-packages/detectron2/'
detectron_base = base 

# for instance segmentation
config_file = os.path.join(detectron_base,'config/quick_schedules/mask_rcnn_R_50_FPN_inference_acc_test.yaml')
opts = ['MODEL.WEIGHTS', 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl', 'MODEL.DEVICE', 'cpu'] 
cfg = setup_cfg(config_file, opts)
demo = VisualizationDemo(cfg)

AssertionError: ignored

In [0]:
# for keypoints detection
config_file = os.path.join(detectron_base,'configs/quick_schedules/keypoint_rcnn_R_50_FPN_inference_acc_test.yaml')
opts = ['MODEL.WEIGHTS', 'detectron2://COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl', 'MODEL.DEVICE', 'cpu']
cfg = setup_cfg(config_file, opts)
demo = VisualizationDemo(cfg)

In [0]:
# for panoptic segmentation
config_file = os.path.join(detectron_base,'configs/quick_schedules/panoptic_fpn_R_50_training_acc_test.yaml')
opts = ['MODEL.WEIGHTS', "detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_50_3x/139514569/model_final_c10459.pkl", 'MODEL.DEVICE', 'cpu']
cfg = setup_cfg(config_file, opts)
demo = VisualizationDemo(cfg)

In [0]:
output = None
video_input = None
webcam = False

mp.set_start_method("spawn", force=True)

In [0]:
cfg = setup_cfg(config_file, opts)
demo = VisualizationDemo(cfg)

In [0]:
inputs = [
#    "/Users/asakawa/Pictures/19960222_002.jpg", 
    "1996Jerusalem_wall_of_tears.png",
#    "/Users/asakawa/Pictures/19960222_004.jpg",
#    "/Users/asakawa/Pictures/19960222_006.jpg",
]
#output = "2020-0426keypoint_detection.jpg"
output = "2020-0426panoptic_segmentation.jpg"
#output = "2020-0426instance_segmentation.jpg"
#output = None

for path in tqdm.tqdm(inputs):
    img = read_image(path, format="BGR")
    start_time = time.time()
    predictions, visualized_output = demo.run_on_image(img)
    if output:
        if os.path.isdir(output):
            assert os.path.isdir(output), output
            out_filename = os.path.join(args.output, os.path.basename(path))
        else:
            assert len(inputs) == 1, "Please specify a directory with output"
            out_filename = output
            visualized_output.save(out_filename)
    else:
        img = (visualized_output.get_image()[:, :, ::-1])
        display(PIL.Image.fromarray(img))


100%|██████████| 1/1 [00:06<00:00,  6.39s/it]


In [0]:
import detectron2.model_zoo as model_zoo

help(model_zoo.get) # config_path, trained: bool = False)
model_zoo.get('../configs/quick_schedules/semantic_R_50_FPN_inference_acc_test.yaml', trained=False)


In [0]:
"""
    CONFIG_PATH_TO_URL_SUFFIX = {
        # COCO Detection with Faster R-CNN
        "COCO-Detection/faster_rcnn_R_50_C4_1x.yaml": "137257644/model_final_721ade.pkl",
        "COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml": "137847829/model_final_51d356.pkl",
        "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml": "137257794/model_final_b275ba.pkl",
        "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml": "137849393/model_final_f97cb7.pkl",
        "COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml": "137849425/model_final_68d202.pkl",
        "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml": "137849458/model_final_280758.pkl",
        "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml": "138204752/model_final_298dad.pkl",
        "COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml": "138204841/model_final_3e0943.pkl",
        "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml": "137851257/model_final_f6e8b1.pkl",
        "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml": "139173657/model_final_68b088.pkl",
        # COCO Detection with RetinaNet
        "COCO-Detection/retinanet_R_50_FPN_1x.yaml": "137593951/model_final_b796dc.pkl",
        "COCO-Detection/retinanet_R_50_FPN_3x.yaml": "137849486/model_final_4cafe0.pkl",
        "COCO-Detection/retinanet_R_101_FPN_3x.yaml": "138363263/model_final_59f53c.pkl",
        # COCO Detection with RPN and Fast R-CNN
        "COCO-Detection/rpn_R_50_C4_1x.yaml": "137258005/model_final_450694.pkl",
        "COCO-Detection/rpn_R_50_FPN_1x.yaml": "137258492/model_final_02ce48.pkl",
        "COCO-Detection/fast_rcnn_R_50_FPN_1x.yaml": "137635226/model_final_e5f7ce.pkl",
        # COCO Instance Segmentation Baselines with Mask R-CNN
        "COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x.yaml": "137259246/model_final_9243eb.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml": "137260150/model_final_4f86c3.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml": "137260431/model_final_a54504.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml": "137849525/model_final_4ce675.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml": "137849551/model_final_84107b.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml": "137849600/model_final_f10217.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml": "138363239/model_final_a2914c.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_101_DC5_3x.yaml": "138363294/model_final_0464b7.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml": "138205316/model_final_a3ec72.pkl",
        "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml": "139653917/model_final_2d9806.pkl",  # noqa
        # COCO Person Keypoint Detection Baselines with Keypoint R-CNN
        "COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml": "137261548/model_final_04e291.pkl",
        "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml": "137849621/model_final_a6e10b.pkl",
        "COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml": "138363331/model_final_997cc7.pkl",
        "COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml": "139686956/model_final_5ad38f.pkl",
        # COCO Panoptic Segmentation Baselines with Panoptic FPN
        "COCO-PanopticSegmentation/panoptic_fpn_R_50_1x.yaml": "139514544/model_final_dbfeb4.pkl",
        "COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml": "139514569/model_final_c10459.pkl",
        "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml": "139514519/model_final_cafdb1.pkl",
        # LVIS Instance Segmentation Baselines with Mask R-CNN
        "LVIS-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml": "144219072/model_final_571f7c.pkl",
        "LVIS-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml": "144219035/model_final_824ab5.pkl",
        "LVIS-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_1x.yaml": "144219108/model_final_5e3439.pkl",  # noqa
        # Cityscapes & Pascal VOC Baselines
        "Cityscapes/mask_rcnn_R_50_FPN.yaml": "142423278/model_final_af9cf5.pkl",
        "PascalVOC-Detection/faster_rcnn_R_50_C4.yaml": "142202221/model_final_b1acc2.pkl",
        # Other Settings
        "Misc/mask_rcnn_R_50_FPN_1x_dconv_c3-c5.yaml": "138602867/model_final_65c703.pkl",
        "Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml": "144998336/model_final_821d0b.pkl",
        "Misc/cascade_mask_rcnn_R_50_FPN_1x.yaml": "138602847/model_final_e9d89b.pkl",
        "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml": "144998488/model_final_480dd8.pkl",
        "Misc/mask_rcnn_R_50_FPN_3x_syncbn.yaml": "169527823/model_final_3b3c51.pkl",
        "Misc/mask_rcnn_R_50_FPN_3x_gn.yaml": "138602888/model_final_dc5d9e.pkl",
        "Misc/scratch_mask_rcnn_R_50_FPN_3x_gn.yaml": "138602908/model_final_01ca85.pkl",
        "Misc/panoptic_fpn_R_101_dconv_cascade_gn_3x.yaml": "139797668/model_final_be35db.pkl",
        "Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml": "18131413/model_0039999_e76410.pkl",  # noqa
        # D1 Comparisons
        "Detectron1-Comparisons/faster_rcnn_R_50_FPN_noaug_1x.yaml": "137781054/model_final_7ab50c.pkl",  # noqa
"""

```bash
ls ../configs/quick_schedules 
README.md                                                 mask_rcnn_R_50_FPN_inference_acc_test.yaml
fast_rcnn_R_50_FPN_inference_acc_test.yaml                mask_rcnn_R_50_FPN_instant_test.yaml
fast_rcnn_R_50_FPN_instant_test.yaml                      mask_rcnn_R_50_FPN_training_acc_test.yaml
keypoint_rcnn_R_50_FPN_inference_acc_test.yaml            panoptic_fpn_R_50_inference_acc_test.yaml
keypoint_rcnn_R_50_FPN_instant_test.yaml                  panoptic_fpn_R_50_instant_test.yaml
keypoint_rcnn_R_50_FPN_normalized_training_acc_test.yaml  panoptic_fpn_R_50_training_acc_test.yaml
keypoint_rcnn_R_50_FPN_training_acc_test.yaml             retinanet_R_50_FPN_inference_acc_test.yaml
mask_rcnn_R_50_C4_GCN_instant_test.yaml                   retinanet_R_50_FPN_instant_test.yaml
mask_rcnn_R_50_C4_GCV_instant_test.yaml                   rpn_R_50_FPN_inference_acc_test.yaml
mask_rcnn_R_50_C4_inference_acc_test.yaml                 rpn_R_50_FPN_instant_test.yaml
mask_rcnn_R_50_C4_instant_test.yaml                       semantic_R_50_FPN_inference_acc_test.yaml
mask_rcnn_R_50_C4_training_acc_test.yaml                  semantic_R_50_FPN_instant_test.yaml
mask_rcnn_R_50_DC5_inference_acc_test.yaml                semantic_R_50_FPN_training_acc_test.yaml
```